In [2]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from collections import Counter
%reload_ext autoreload
%autoreload 2
from utils import *
plt.style.use("fivethirtyeight")
import pickle

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict, GridSearchCV,RepeatedStratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, classification_report, plot_confusion_matrix, auc, mean_squared_error, confusion_matrix, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import resample
from imblearn.under_sampling import CondensedNearestNeighbour, NearMiss, OneSidedSelection, NeighbourhoodCleaningRule, RandomUnderSampler, TomekLinks
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.ensemble import BalancedBaggingClassifier, BalancedRandomForestClassifier, EasyEnsembleClassifier, RUSBoostClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.dummy import DummyClassifier
from imblearn.pipeline import Pipeline
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [4]:
pickle_in = open("data/training_model.pickle","rb")
train = pickle.load(pickle_in)
pickle_in = open("data/validate_model.pickle","rb")
validate = pickle.load(pickle_in)

In [5]:
X_train = train.drop(["default"], axis=1)
y_tr = train["default"]
X_validate = validate.drop(["default"], axis=1)
y_val = validate["default"]

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
X_tr = scaler.transform(X_train)
X_val = scaler.transform(X_validate)

In [7]:
dc = DummyClassifier(strategy='most_frequent').fit(X_tr, y_tr)
y_pred_dc_tr = dc.predict(X_tr)
y_pred_dc_val = dc.predict(X_val)
get_metric(X_tr, y_tr, X_val, y_val, y_pred_dc_tr, y_pred_dc_val, dc)

Training Accuracy:  0.7782857142857142
Validation Accuracy:  0.7811666666666667
Training F1 Score:  0.0
Validation F1 Score:  0.0
Training AUC Score:  0.5
Validation AUC Score:  0.5
Training Recall Score:  0.0
Validation Recall Score:  0.0
Training Precision Score:  0.0
Validation Precision Score:  0.0
Training Average Precision Score:  0.22171428571428572
Validation Average Precision Score:  0.21883333333333332


In [8]:
bc = BaggingClassifier(n_estimators=50, random_state=42).fit(X_tr, y_tr)
y_pred_bc_tr = bc.predict(X_tr)
y_pred_bc_val = bc.predict(X_val)
get_metric(X_tr, y_tr, X_val, y_val, y_pred_bc_tr, y_pred_bc_val, bc)
print("")
print('Bagging Classifier Performance:')
print('Balanced training accuracy: {:.2f} - Geometric mean {:.2f}'.format(balanced_accuracy_score(y_tr, y_pred_bc_tr), geometric_mean_score(y_tr, y_pred_bc_tr)))
print('Balanced validation accuracy: {:.2f} - Geometric mean {:.2f}'.format(balanced_accuracy_score(y_val, y_pred_bc_val), geometric_mean_score(y_val, y_pred_bc_val)))

Training Accuracy:  0.996904761904762
Validation Accuracy:  0.807
Training F1 Score:  0.9929964443486693
Validation F1 Score:  0.44326923076923075
Training AUC Score:  0.9998954304300327
Validation AUC Score:  0.740359286457933
Training Recall Score:  0.9896907216494846
Validation Recall Score:  0.3511043412033511
Training Precision Score:  0.9963243243243243
Validation Precision Score:  0.6010430247718384
Training Average Precision Score:  0.9996190235446103
Validation Average Precision Score:  0.48114820511254575

Bagging Classifier Performance:
Balanced training accuracy: 0.99 - Geometric mean 0.99
Balanced validation accuracy: 0.64 - Geometric mean 0.57


In [ ]:
bc = BaggingClassifier(base_estimator)

In [9]:
bbc = BalancedBaggingClassifier(n_estimators=50, random_state=42).fit(X_tr, y_tr)
y_pred_bbc_tr = bbc.predict(X_tr)
y_pred_bbc_val = bbc.predict(X_val)
get_metric(X_tr, y_tr, X_val, y_val, y_pred_bbc_tr, y_pred_bbc_val, bbc)
print("")
print('Balanced Bagging Classifier Performance:')
print('Balanced training accuracy: {:.2f} - Geometric mean {:.2f}'.format(balanced_accuracy_score(y_tr, y_pred_bbc_tr), geometric_mean_score(y_val, y_pred_bbc_tr)))
print('Balanced validation accuracy: {:.2f} - Geometric mean {:.2f}'.format(balanced_accuracy_score(y_val, y_pred_bbc_val), geometric_mean_score(y_val, y_pred_bbc_val)))

Training Accuracy:  0.9383333333333334
Validation Accuracy:  0.7631666666666667
Training F1 Score:  0.8778186621379375
Validation F1 Score:  0.5111799105607154
Training AUC Score:  0.9955504481714446
Validation AUC Score:  0.7528188434539899
Training Recall Score:  0.9991408934707904
Validation Recall Score:  0.5658796648895659
Training Precision Score:  0.7827696449604576
Validation Precision Score:  0.46612296110414053
Training Average Precision Score:  0.9819759873453777
Validation Average Precision Score:  0.499848233823849

Balanced Bagging Classifier Performance:
Balanced training accuracy: 0.96 - Geometric mean 0.00
Balanced validation accuracy: 0.69 - Geometric mean 0.68


In [11]:
pickle_in = open("data/best_model.pickle","rb")
rfcb = pickle.load(pickle_in)

In [12]:
tl = TomekLinks()
sampling(X_tr, y_tr, X_val, y_val, tl, rfcb)

Training Count:  Counter({0: 14844, 1: 4656})
Validation Count:  Counter({0: 4271, 1: 1313})
Training Accuracy:  0.839948717948718
Validation Accuracy:  0.8216332378223495
Training F1 Score:  0.5707605556319626
Validation F1 Score:  0.508390918065153
Training AUC Score:  0.841091632763096
Validation AUC Score:  0.7968306061015121
Training Recall Score:  0.44566151202749144
Validation Recall Score:  0.39223153084539225
Training Precision Score:  0.7934990439770554
Validation Precision Score:  0.7223001402524544
Training Average Precision Score:  0.7049875136912391
Validation Average Precision Score:  0.6050182959215592


In [13]:
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours()
sampling(X_tr, y_tr, X_val, y_val, enn, rfcb)

Training Count:  Counter({0: 9921, 1: 4656})
Validation Count:  Counter({0: 2811, 1: 1313})
Training Accuracy:  0.8493517184605885
Validation Accuracy:  0.8208050436469447
Training F1 Score:  0.7141369435042958
Validation F1 Score:  0.6630186958504332
Training AUC Score:  0.893195115986742
Validation AUC Score:  0.848395881374526
Training Recall Score:  0.5891323024054983
Validation Recall Score:  0.5536938309215537
Training Precision Score:  0.9064771976206213
Validation Precision Score:  0.8261363636363637
Training Average Precision Score:  0.8544751985887381
Validation Average Precision Score:  0.7911518345139913


In [14]:
sm = SMOTE(sampling_strategy='minority', random_state=42)
sampling(X_tr, y_tr, X_val, y_val, sm, rfcb)

Training Count:  Counter({0: 16344, 1: 16344})
Validation Count:  Counter({0: 4687, 1: 4687})
Training Accuracy:  0.7620533529123837
Validation Accuracy:  0.7224237251973544
Training F1 Score:  0.7464302014735605
Validation F1 Score:  0.6975122064636132
Training AUC Score:  0.8463959109595244
Validation AUC Score:  0.8030891021377533
Training Recall Score:  0.7004405286343612
Validation Recall Score:  0.6400682739492213
Training Precision Score:  0.7988834612700628
Validation Precision Score:  0.7662835249042146
Training Average Precision Score:  0.8526563021371741
Validation Average Precision Score:  0.810826989470193


In [18]:
from imblearn.over_sampling import SVMSMOTE
svmsm = SVMSMOTE()
sampling(X_tr, y_tr, X_val, y_val, svmsm, rfcb)

Training Count:  Counter({0: 16344, 1: 16344})
Validation Count:  Counter({0: 4687, 1: 4687})
Training Accuracy:  0.781754772393539
Validation Accuracy:  0.7906976744186046
Training F1 Score:  0.7695736434108527
Validation F1 Score:  0.7846322722283205
Training AUC Score:  0.8678097486426797
Validation AUC Score:  0.8629364644496721
Training Recall Score:  0.7288913362701909
Validation Recall Score:  0.7625346703648389
Training Precision Score:  0.8150656814449918
Validation Precision Score:  0.808048835631924
Training Average Precision Score:  0.8754784149639834
Validation Average Precision Score:  0.8728034854708415


In [15]:
from imblearn.over_sampling import ADASYN
adsn = ADASYN()
sampling(X_tr, y_tr, X_val, y_val, adsn, rfcb)

Training Count:  Counter({1: 16573, 0: 16344})
Validation Count:  Counter({0: 4687, 1: 4536})
Training Accuracy:  0.7403469331956132
Validation Accuracy:  0.6970616935921067
Training F1 Score:  0.736374572036643
Validation F1 Score:  0.6800274851122309
Training AUC Score:  0.8150627820568924
Validation AUC Score:  0.7682545985387178
Training Recall Score:  0.7202679056296386
Validation Recall Score:  0.6545414462081128
Training Precision Score:  0.7532180716809692
Validation Precision Score:  0.7075786463298379
Training Average Precision Score:  0.8167019672323599
Validation Average Precision Score:  0.7678434827387284


In [16]:
from imblearn.combine import SMOTETomek
smtk = SMOTETomek()
sampling(X_tr, y_tr, X_val, y_val, smtk, rfcb)

Training Count:  Counter({0: 15617, 1: 15617})
Validation Count:  Counter({0: 4448, 1: 4448})
Training Accuracy:  0.7696740731254402
Validation Accuracy:  0.7419064748201439
Training F1 Score:  0.7534950657894737
Validation F1 Score:  0.7223028543783261
Training AUC Score:  0.8562297616043882
Validation AUC Score:  0.8201241474221702
Training Recall Score:  0.7040404687199846
Validation Recall Score:  0.6713129496402878
Training Precision Score:  0.8104223483452495
Validation Precision Score:  0.7816753926701571
Training Average Precision Score:  0.8615989775670404
Validation Average Precision Score:  0.8256320993928399


In [17]:
from imblearn.combine import SMOTEENN
smenn = SMOTEENN(sampling_strategy="minority", n_jobs= -1)
sampling(X_tr, y_tr, X_val, y_val, smenn, rfcb)

Training Count:  Counter({1: 11465, 0: 8660})
Validation Count:  Counter({1: 3249, 0: 2360})
Training Accuracy:  0.8723478260869565
Validation Accuracy:  0.8111962916740952
Training F1 Score:  0.8839918717543463
Validation F1 Score:  0.8285575522098108
Training AUC Score:  0.9484440797325729
Validation AUC Score:  0.905861255875341
Training Recall Score:  0.8537287396423899
Validation Recall Score:  0.7876269621421976
Training Precision Score:  0.9164794007490636
Validation Precision Score:  0.8739754098360656
Training Average Precision Score:  0.9653497494543584
Validation Average Precision Score:  0.9379111317969824
